<a href="https://colab.research.google.com/github/Mtlukasik/Exploration/blob/main/OOD_12022024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

- Import additional libraries of your choice.

Although you are expected to demonstrate understanding of ML/DS/statistics tools, a particular choice of libraries and frameworks will not affect evaluation of the solution.

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
def check_calibration(dataset,predicted_probs):
    x_vals = []
    y_vals = []
    # Since we can't directly iterate i and j in pairs with range, let's define j as i + step within the loop.
    step = 0.01
    for i in np.arange(0, 1, step):
      try:
          j = i + step  # Defining j based on the step
          # Finding indices where probs meet the condition between i and j
          indices = np.where(np.logical_and(predicted_probs.flatten() >= i, predicted_probs.flatten() <= j))[0]
          a = dataset['install'].values[indices]
          if len(a)>0:
            x_vals.append((i+j)/2)
            y_vals.append(sum(a)/len(a))
      except ZeroDivisionError:
        continue
    # Plotting
    plt.plot(x_vals,x_vals)
    plt.scatter(x_vals, y_vals)
    plt.xlabel('(i + j) / 2')
    plt.ylabel('Mean of Selected Install Values')
    plt.title('Scatter Plot of Mean Install Values')
    plt.show()
#check_calibration(TRAIN_DATA,probs)

class OODModel:

    def __init__(self, input_shape):
        self.model = self._build_model(input_shape)
        self.target_column = 'OOD'

    def _build_model(self, input_shape):
        # Define a simple Sequential model
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(1, activation='sigmoid')  # Assuming binary classification
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def fit(self, X, y, epochs=2, validation_split=0.2):
        self.model.fit(X, y, epochs=epochs, validation_split=validation_split)

    def predict(self, X):
        return self.model.predict(X)

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
import keras
import sklearn
import hashlib
import matplotlib.pyplot as plt
import scipy.stats as stats
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Model
from keras.layers import Input, Dense, Reshape
# Import additional libraries of your choice

# Unity Data Science quiz

At Unity, we develop deep learning models for real-time ads bidding ([OpenRTB](https://www.iab.com/guidelines/openrtb/)) at various ad
exchanges. To bid for an ad impression, we estimate the optimal bid value using predicted
install probability of campaign together with several other factors e.g. cost per install.

In this homework, your task is to **train a model to predict *install probabilities* for ad impressions included in
the test data**. In the production environment, the model predictions are used for deriving the optimial bids for available ad campaigns.  The best ad campaign will be shown to the user. Overestimation of install probabilities will lead to unnecessarily high bids and monetary losses, while underestimation of install probabilities will lead to unnecessarily low bids and loss of opportunities for Unity to win ad impressions. Therefore, it is important for the model predictions to be as accurate as possible.

## Instructions

- Complete the homework using Python and libraries of your choice.
- Follow the instructions in this notebook.
- Keep the code clean and organized.

## Evaluation

We focus our evaluation on technical proficiency, analytical skills, problem solving, creative thinking as well as ability to communicate clearly. In particular, we will evaluate:
- understanding of the problem (e.g. does a delivered solution meet the specification of the task)
- quality of discussion and brevity of the report (e.g. comments in this notebook)
- quality of EDA
- feature handling & preprocessing
- modeling approach
- model validation and evaluation

We will separately evaluate the predicted test set install probabilities (see below). Although considered as part of the evaluation, the final performance is not the key factor and shall not dominate over the above dimensions.

## Data description

- ```id```: impression id
- ```timestamp```: time of the event in UTC ```
 all installs happened  long ago this game is probably old```
- ```campaignId```: id of the advertising campaign (the game being advertised)
- ```platform```: device platform
- ```softwareVersion```: OS version of the device
- ```country```: country of user
- ```sourceGameId```: id of the publishing game (the game being played)
- ```startCount```: how many times the user has started (any) campaigns
- ```viewCount```: how many times the user has viewed (any) campaigns
- ```clickCount```: how many times the user has clicked (any) campaigns
- ```installCount```: how many times the user has installed games from this ad network
- ```lastStart```: last time the user started any campaign
- ```startCount1d```: how many times the user has started (any) campaigns within the last 24 hours
- ```startCount7d```: how many times the user has started (any) campaigns within the last 7 days
- ```connectionType```: internet connection type
- ```deviceType```: device model
- ```install```: binary indicator if install was observed (install=1) or not (install=0) after impression

## Submission

- This Jupyter notebook
- A CSV file containing the predicted install probabilities of ad impressions in the test data. The file should have the following columns:
    - ```id```: ID of ad impression in the test data
    - ```install_proba```: Predicted install probability of ad impression


## Load and prepare data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_df = pd.read_csv("/content/drive/MyDrive/training_data.csv", sep=";", parse_dates=True)
train_df = train_df[~train_df['install'].isna()]#inputing install label is classification so there is no use in it
train_df = train_df[~train_df['lastStart'].isna()]
train_df['lastStart'] = pd.to_datetime(train_df['lastStart'])
train_df = train_df.sort_values(by=['timestamp']).reset_index(drop=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
# Ensure 'timestamp' is in datetime format
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
daily_groups = train_df.groupby(pd.Grouper(key='timestamp', freq='D'))
daily_dfs = []
for _, group in daily_groups:
    if not group.empty:  # Check if the group is not empty
        daily_dfs.append(group)
[i.shape for i in daily_dfs]

[(315482, 17),
 (313690, 17),
 (306825, 17),
 (281678, 17),
 (232467, 17),
 (225278, 17),
 (189106, 17),
 (201060, 17),
 (214095, 17),
 (211327, 17),
 (223277, 17),
 (257102, 17),
 (272655, 17),
 (219667, 17)]

In [ ]:
daily_unseen_dfs = [df[df['country'] == 'US'] for df in daily_dfs[:7]]
daily_modified_dfs = []

for df in daily_dfs[:7]:
    # Remove rows where 'country' == 'US' and add to the new list
    daily_modified_dfs.append(df[df['country'] != 'US'])

In [ ]:
seen_df = pd.concat(daily_modified_dfs)
unseen_df = pd.concat(daily_unseen_dfs)

In [ ]:
TRAIN_DATA = seen_df
UNSEEN_DATA = unseen_df
TRAFFIC_DATA = daily_dfs[8]

In [ ]:
TRAIN_DATA.head()

,id,timestamp,campaignId,platform,softwareVersion,sourceGameId,country,startCount,viewCount,clickCount,installCount,lastStart,startCount1d,startCount7d,connectionType,deviceType,install
0,5c33e8004eeff537f7cab9af,2019-01-08 00:00:00.368000+00:00,5c26e5222127049b4bce982d,android,8.0.0,1432611,NZ,44,41,0,2,2019-01-07 20:50:38.893000+00:00,5,12,wifi,samsung SM-G935F,0
2,5c33e8005b62743723aab607,2019-01-08 00:00:00.752000+00:00,5be28ecbc621317c947a63c7,ios,12.1.2,2612974,FR,129,124,1,1,2019-01-07 23:59:27.976000+00:00,6,31,wifi,"iPhone10,6",0
3,5c33e800ffa6e3378d7b4ffd,2019-01-08 00:00:00.795000+00:00,5af41f7dce114315dc22d94d,android,6.0.1,2626028,BR,11,7,0,0,2019-01-07 13:19:03.738000+00:00,4,4,wifi,samsung SM-J500M,0
5,5c33e801faa1d437f73878a9,2019-01-08 00:00:01.857000+00:00,5b42ff00354f8159b6a49b15,android,6.0.1,61378,AR,21,13,2,0,2019-01-07 23:59:25.936000+00:00,4,19,wifi,samsung SM-G532M,0
6,5c33e8026922e637c2ceae6e,2019-01-08 00:00:02.094000+00:00,5c2ed1c592f69c5f56a4c852,ios,12.1.2,20721,MK,41,40,3,0,2019-01-07 23:57:47.384000+00:00,2,15,wifi,"iPhone10,2",0


In [ ]:
from typing import Optional

class SampleWeightNormalizer:
    @staticmethod
    def reweight(sample_weight: Optional[tf.Tensor]) -> Optional[tf.Tensor]:

        if sample_weight is not None:
            sample_weight = (
                sample_weight
                * tf.cast(tf.shape(sample_weight)[0], tf.float32)
                / tf.reduce_sum(sample_weight, axis=0)
            )

        return sample_weight
@tf.keras.utils.register_keras_serializable()
class WeightedBCELossWrapper(tf.keras.losses.BinaryCrossentropy, SampleWeightNormalizer):
    def __init__(self, y_pred_idx=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # ensure only the sampled branch is used in loss calculation
        # y_pred_idx variable specifies the index of the branch (mean or sampled) to be used in the loss wrapper
        self.y_pred_idx = y_pred_idx

    def __call__(
        self,
        y_true: tf.Tensor,
        y_pred: tf.Tensor,
        sample_weight: Optional[tf.Tensor] = None,
    ) -> Optional[tf.Tensor]:

        if self.y_pred_idx is not None:
            y_pred = tf.gather(y_pred, self.y_pred_idx, axis=-1)

        return super().__call__(
            y_true=y_true,
            y_pred=y_pred,
            sample_weight=self.reweight(sample_weight),
        )

    def get_config(self):
        """Method that returns a dictionary with its key-value pair representing the kwargs in `self`"""
        return {}

@tf.keras.utils.register_keras_serializable()
class WeightedBCEMetricWrapper(tf.keras.metrics.BinaryCrossentropy, SampleWeightNormalizer):

    def __init__(self, y_pred_idx=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # ensure only the sampled branch is used in loss metric calculation
        self.y_pred_idx = y_pred_idx

    def update_state(
        self,
        y_true: tf.Tensor,
        y_pred: tf.Tensor,
        sample_weight: Optional[tf.Tensor] = None,
    ) -> Optional[tf.Tensor]:

        if self.y_pred_idx is not None:
            y_pred = tf.gather(y_pred, self.y_pred_idx, axis=-1)

        return super().update_state(
            y_true=y_true,
            y_pred=y_pred,
            sample_weight=self.reweight(sample_weight),
        )

    def get_config(self):
        """Method that returns a dictionary with its key-value pair representing the kwargs in `self`"""
        return {}
import pandas as pd
from sklearn.utils import resample


class DataPreprocessor:

    def __init__(self, numerical_columns, target_column, categorical_columns=None):
        self.numerical_columns = numerical_columns
        self.target_column = target_column
        # Initialize with an empty list if no categorical columns are provided
        self.categorical_columns = categorical_columns if categorical_columns is not None else []
        # Initialize mappings only if there are categorical columns
        self.cat_col_mappings = {col: {} for col in self.categorical_columns}

    def fit_categorical_mappings(self, df):
        # Proceed only if there are categorical columns
        if self.categorical_columns:
            for col in self.categorical_columns:
                value_counts = df[col].value_counts()
                valid_values = value_counts[value_counts > 100].index.tolist()
                self.cat_col_mappings[col] = {v: i+1 for i, v in enumerate(valid_values)}

    def map_categorical_values(self, df):
        # Map values only if there are categorical columns
        if self.categorical_columns:
            for col in self.categorical_columns:
                df[col] = df[col].apply(lambda x: self.cat_col_mappings[col].get(x, 0))
        return df

    def prepreprocess_(self, df):
        self.fit_categorical_mappings(df)

        # Include categorical columns in processing only if they exist
        columns = self.numerical_columns + (self.categorical_columns if self.categorical_columns else [])
        if self.target_column in df.columns:
            columns += [self.target_column]
        processed_df = df[columns].copy()

        # Convert numerical columns to float
        for col in self.numerical_columns:
            processed_df[col] = pd.to_numeric(processed_df[col], errors='coerce').astype(float)

        # Map categorical columns to IDs, if any
        processed_df = self.map_categorical_values(processed_df)
        return processed_df

    def preprocess_(self, df, return_xy=True, fit_cat_mappings=False):
        processed_df = self.prepreprocess_(df)

        if return_xy:
            processed_df, y, weights = self.preprocess_train(processed_df)
        # Include categorical columns in X_train if they exist
        X_train = {col + "_input": processed_df[col].values for col in self.numerical_columns+self.categorical_columns}

        if return_xy:
            return X_train, y, weights
        else:
            return X_train

    # Your existing preprocess_train method remains unchanged
    def preprocess_train(self, df):
        # Existing implementation

        label_counts = df[self.target_column].value_counts()
        majority_label = label_counts.idxmax()
        minority_label = label_counts.idxmin()

        df_majority = df[df[self.target_column] == majority_label]
        df_minority = df[df[self.target_column] == minority_label]

        df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=123)
        balanced_df = pd.concat([df_minority, df_majority_downsampled])
        balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

        # Now, you can convert this balanced_df into X_train, y format or continue using it as a DataFrame for further processing.
        X_train = balanced_df[balanced_df.columns.difference([self.target_column])]
        y = pd.to_numeric(balanced_df[self.target_column], errors='coerce').astype('float32').values
        sampling_ratio = len(df_minority) / (len(df_minority) + len(df_majority))
        weights = balanced_df[self.target_column].apply(
            lambda x: 1 if x == minority_label else 1 / sampling_ratio
        ).values

        return X_train, y, weights


@tf.keras.utils.register_keras_serializable(package="Custom", name="PermanentDropout")
class PermanentDropout(tf.keras.layers.Dropout):
    def call(self, inputs, training=None):
        return super().call(inputs, training=True)

class ConversionModel:

    def __init__(self, input_shape,numerical_columns,categorical_columns,cat_col_mappings):
        self.continuous_feats = numerical_columns
        self.categorical_feats = categorical_columns
        self.target_column = 'install'
        self.CPILoader = types.SimpleNamespace()
        self.CPILoader.cat_col_mappings = cat_col_mappings

        self.neural_network_structure = {   'add_batchnorm': True, 'input_dropout': 0.0,'n_layers': 3,
            '0':{'activation': 'swish',
              'size': 512/8,
              'dropout': 0.2},
            '1':{'activation': 'swish',
              'size': 256/8,
              'dropout': 0.1},
            '2':{'activation': 'swish',
              'size': 128/8,
              'dropout': 0.06},
            'hidden_l1': 0.0,
            'hidden_l2': 0.0}
        self.model = self._build_model(input_shape)
    def build_dnn_two_towers_with_shared_weights(self):
          """
          creates 2 identical towers with shared weights
          one of the towers is used for mean prediction, the other for sampling
          the mc dropout is applied only to the sampled tower
          """
          mean_dnn_layers = []
          sampling_dnn_layers = []
          for i in range(self.neural_network_structure["n_layers"]):
              config = self.neural_network_structure[str(i)]

              dense_units = config["size"]
              activation = config["activation"]
              dropout_rate = config["dropout"]


              dense_layer = Dense(
                  dense_units,
                  kernel_regularizer=tf.keras.regularizers.L1L2(
                      l1=self.neural_network_structure.get("hidden_l1", 0.0),
                      l2=self.neural_network_structure.get("hidden_l2", 0.0),
                  ),
                  name=f"hidden_{i}",
              )

              layers = [
                  dense_layer,
                  tf.keras.layers.BatchNormalization() if self.neural_network_structure.get("add_batchnorm") else None,
                  tf.keras.layers.Activation(activation),
              ]

              mean_dnn_layers += [k for k in layers if k is not None]+ [tf.keras.layers.Dropout(dropout_rate)]
              sampling_dnn_layers += [k for k in layers if k is not None] + [PermanentDropout(dropout_rate)]

          return (
              tf.keras.models.Sequential(mean_dnn_layers, name="mean_deep_neural_network"),
              tf.keras.models.Sequential(sampling_dnn_layers, name="sampling_deep_neural_network"),
              )
    def embedding_input(self,
        name,
        n_in,
        n_out,
        initializer_name="truncated_normal",
        l1=0.0,
        l2=0.0,
    ):
        inp = Input(shape=(1,), dtype="int64", name=name + "_input")

        if initializer_name == "uniform":
            initializer = tf.keras.initializers.RandomUniform(minval=-1.0, maxval=1.0)
        elif initializer_name == "truncated_normal":
            initializer = tf.keras.initializers.TruncatedNormal(mean=0, stddev=1 / np.sqrt(n_in))
        else:
            raise ValueError("ERROR: Trying to initialize embeddings with unknown initializer.")

        regularizer = tf.keras.regularizers.L1L2(l1=l1, l2=l2) if l1 > 0.0 or l2 > 0.0 else None

        return inp, tf.keras.layers.Embedding(
            n_in,
            n_out,
            input_length=1,
            embeddings_initializer=initializer,
            embeddings_regularizer=regularizer,
            name=name + "_embedding",
        )(inp)

    def build_inputs(self,category_mappings):
        def continuous_input(name, preprocessed_key=None):
          input_name = preprocessed_key or name + "_input"
          inp = Input(shape=(1,), dtype="float32", name=input_name)
          return inp, Reshape((1, 1), name=name + "_reshape")(inp)
        inp_layer = []
        inp_embed = []
        if self.categorical_feats:
          for cat_col in self.categorical_feats:
              embed_dim = 10
              n_in = len(category_mappings[cat_col])+1
              t_inp, t_build = self.embedding_input(
                  cat_col,n_in,n_out=embed_dim)
              if self.neural_network_structure.get("add_batchnorm"):
                t_build = tf.keras.layers.BatchNormalization()(t_build)

              inp_layer.append(t_inp)
              inp_embed.append(t_build)
        for con_col in self.continuous_feats:
            t_inp, t_build = continuous_input(con_col)
            inp_layer.append(t_inp)
            inp_embed.append(t_build)
        return inp_layer, inp_embed

    def _build_model(self, input_shape):
        input_layer, inputs = self.build_inputs(self.CPILoader.cat_col_mappings)
        #print(f"Concatenated Inputs Shape: {len(inputs)},{inputs[0].shape},{inputs[1].shape}")

        #print(self.CPILoader.cat_col_mappings)
        #concatenated_inputs = tf.keras.layers.Concatenate(axis=-1)(inputs)
        flattened_inputs = tf.keras.layers.Flatten()(tf.keras.layers.concatenate(inputs))
        x = tf.keras.layers.Dropout(self.neural_network_structure["input_dropout"])(flattened_inputs)

        mean_dnn, sampling_dnn = self.build_dnn_two_towers_with_shared_weights()
        mean, sampled = mean_dnn(x), sampling_dnn(x)
        output_layer = Dense(1, activation="sigmoid", name="sample")
        mean, sampled = output_layer(mean), output_layer(sampled)
        outputs = tf.keras.layers.concatenate([mean, sampled], name="output")
        #print(f"Output Shape after DNN: {outputs.shape}")
        self.model = tf.keras.models.Model(input_layer, outputs, name="model")
        self.model.compile(optimizer='adam', loss=WeightedBCELossWrapper(),
            weighted_metrics=[WeightedBCEMetricWrapper()])
        #self.model.summary()
        return self.model
    def fit(self, X, y, sample_weight=None, epochs=30, validation_split=0.2):
        self.model.fit(X, y, sample_weight=None, epochs=epochs, validation_split=0.2, callbacks=[PrintFirstBatchCallback()])


    def predict(self, X):
        return self.model.predict(X)

class PrintFirstBatchCallback(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        if batch == 0:  # Check if it's the first batch
            # Accessing model inputs directly, adjust as necessary for your model structure
            inputs = self.model.inputs
            tf.print("First batch inputs:", inputs, summarize=-1)  # summarize=-1 prints all elements


class Postprocessor:
    def __init__(self, model):
        self.model = model

    def predict(self, X):
        # Get the model's prediction probabilities
        self.predictions = self.model.predict(X)

        # Apply the adjusted threshold to these probabilities to get binary predictions
        binary_predictions = (self.predictions > 0.5).astype(int)
        return binary_predictions, self.predictions

In [ ]:
class Valuator:
    def __init__(self,preprocessor,model,postprocessor):
        #TODO: implement adjusted_threshold here
        self.preprocessor=preprocessor
        self.model = model
        self.postprocessor = postprocessor

    def predict(self, data):
        # Preprocess data to get it in the correct format
        preprocessed_data = self.preprocessor.preprocess_(data, return_xy = False) # Ignore y during prediction

        # Ensure X is a list of arrays matching the model's input structure

        return self.postprocessor.predict(preprocessed_data)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
numerical_columns = ['startCount', 'viewCount', 'clickCount', 'installCount', 'startCount1d', 'startCount7d']
categorical_columns = ['platform','campaignId','sourceGameId']
target_column = 'install'
def given_data_create_valuator(daily_df,target_column,epochs=20):
  # Assuming 'install' is your target column

  # Initialize the preprocessor
  #def given_data_create_valuator(daily_df,target_column):
  preprocessor = DataPreprocessor(numerical_columns=numerical_columns, target_column=target_column,categorical_columns=categorical_columns)
  X,y,weights = preprocessor.preprocess_(daily_df,return_xy=True)
  # Convert the target to a numeric type, if it's not already
  input_shape = pd.DataFrame(X).shape[1]
  # Assuming X is your training features DataFrame and you have 6 numerical columns
  my_model = ConversionModel(input_shape, numerical_columns,categorical_columns,preprocessor.cat_col_mappings)
  my_model.model.fit(X, y, sample_weight=weights, epochs=epochs, validation_split=0.2)
  postprocessor = Postprocessor(my_model)
  return Valuator(preprocessor,my_model,postprocessor)

In [ ]:
from sklearn.utils import resample
valuator = given_data_create_valuator(TRAIN_DATA,'install',epochs=20)

Epoch 1/20
882/882 [==============================] - 10s 7ms/step - loss: 0.1852 - binary_crossentropy: 0.1852 - val_loss: 0.0678 - val_binary_crossentropy: 0.0678
Epoch 2/20
882/882 [==============================] - 5s 6ms/step - loss: 0.0639 - binary_crossentropy: 0.0639 - val_loss: 0.0630 - val_binary_crossentropy: 0.0630
Epoch 3/20
882/882 [==============================] - 4s 5ms/step - loss: 0.0618 - binary_crossentropy: 0.0618 - val_loss: 0.0632 - val_binary_crossentropy: 0.0632
Epoch 4/20
882/882 [==============================] - 5s 6ms/step - loss: 0.0604 - binary_crossentropy: 0.0604 - val_loss: 0.0639 - val_binary_crossentropy: 0.0639
Epoch 5/20
882/882 [==============================] - 4s 5ms/step - loss: 0.0591 - binary_crossentropy: 0.0591 - val_loss: 0.0633 - val_binary_crossentropy: 0.0633
Epoch 6/20
882/882 [==============================] - 4s 5ms/step - loss: 0.0583 - binary_crossentropy: 0.0583 - val_loss: 0.0639 - val_binary_crossentropy: 0.0639
Epoch 7/20
882/

In [ ]:
aquired_data,probs = valuator.predict(TRAFFIC_DATA)

6691/6691 [==============================] - 17s 2ms/step


In [ ]:
unseen_data_ = UNSEEN_DATA.drop(columns=['install'])
train_data_ = TRAIN_DATA.drop(columns=['install'])
# Add OOD column
unseen_data_['OOD'] = 1
train_data_['OOD'] = 0
ood_train_data = pd.concat([train_data_, unseen_data_], ignore_index=True)
ood_train_data = ood_train_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
valuator_ood = given_data_create_valuator(ood_train_data,'OOD',epochs=12)

Epoch 1/12
14647/14647 [==============================] - 97s 6ms/step - loss: 0.1200 - binary_crossentropy: 0.1200 - val_loss: 0.0982 - val_binary_crossentropy: 0.0982
Epoch 2/12
14647/14647 [==============================] - 95s 6ms/step - loss: 0.1001 - binary_crossentropy: 0.1001 - val_loss: 0.0962 - val_binary_crossentropy: 0.0962
Epoch 3/12
14647/14647 [==============================] - 90s 6ms/step - loss: 0.0974 - binary_crossentropy: 0.0974 - val_loss: 0.0956 - val_binary_crossentropy: 0.0956
Epoch 4/12
14647/14647 [==============================] - 89s 6ms/step - loss: 0.0956 - binary_crossentropy: 0.0956 - val_loss: 0.0953 - val_binary_crossentropy: 0.0953
Epoch 5/12
14647/14647 [==============================] - 90s 6ms/step - loss: 0.0944 - binary_crossentropy: 0.0944 - val_loss: 0.0945 - val_binary_crossentropy: 0.0945
Epoch 6/12
14647/14647 [==============================] - 90s 6ms/step - loss: 0.0933 - binary_crossentropy: 0.0933 - val_loss: 0.0931 - val_binary_crossen

2000 x danych niz parameterow rule of thumb

In [ ]:
_,probs_ood = valuator_ood.predict(TRAFFIC_DATA)

6691/6691 [==============================] - 16s 2ms/step


In [ ]:
#Recall@100,Recall@10,Recall@1000
Recall = np.hstack([probs,probs_ood,(TRAFFIC_DATA['country']=='US').map(int).values.reshape(-1,1),TRAFFIC_DATA['install'].map(int).values.reshape(-1,1)])
print(f"Ratio = {sum(Recall[:,4])/len(Recall[:,4])}")
print(f"Recall of IN in MCdropout mean branch: {Recall[Recall[:, 0].argsort()[::-1]][:1000][:,4].sum()}Recall of IN in MCdropout {Recall[Recall[:, 1].argsort()[::-1]][:1000][:,4].sum()},Recall of IN in OOD classifier {Recall[Recall[:, 2].argsort()[::-1]][:1000][:,4].sum()} out of 1000")
print(f"Recall of IN in MCdropout mean branch: {Recall[Recall[:, 0].argsort()[::-1]][:100][:,4].sum()},Recall of IN in MCdropout {Recall[Recall[:, 1].argsort()[::-1]][:100][:,4].sum()},Recall of IN in OOD classifier {Recall[Recall[:, 2].argsort()[::-1]][:100][:,4].sum()} out of 100")
print(f"Recall of IN in MCdropout mean branch: {Recall[Recall[:, 0].argsort()[::-1]][:10][:,4].sum()},RRecall of IN in MCdropout {Recall[Recall[:, 1].argsort()[::-1]][:10][:,4].sum()},Recall of IN in OOD classifier {Recall[Recall[:, 2].argsort()[::-1]][:10][:,4].sum()} out of 10")

Ratio = 0.15059202690394452
Recall of IN in MCdropout mean branch: 198.0Recall of IN in MCdropout 205.0,Recall of IN in OOD classifier 63.0 out of 1000
Recall of IN in MCdropout mean branch: 19.0,Recall of IN in MCdropout 20.0,Recall of IN in OOD classifier 5.0 out of 100
Recall of IN in MCdropout mean branch: 1.0,RRecall of IN in MCdropout 2.0,Recall of IN in OOD classifier 1.0 out of 10


In [ ]:
test_probs = valuator_ood.predict(TRAIN_DATA)

49113/49113 [==============================] - 122s 2ms/step


In [ ]:
test_probs

array([2.0352405e-21, 7.5745680e-21, 8.0354813e-21, ..., 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00], dtype=float32)

In [ ]:
TRAIN_DATA.shape,UNSEEN_DATA.shape,TRAFFIC_DATA.shape

((1571590, 17), (292936, 17), (214095, 17))

In [ ]:
aquired_data,probs = valuator.predict(TRAFFIC_DATA)

6691/6691 [==============================] - 9s 1ms/step


In [ ]:
#let's take top 30% according to the probs
aquired_data = probs.flatten() > np.percentile(probs.flatten(),70)

In [ ]:
def from_valuator_create_ood(which_data_aquired, new_valuator_data, old_training_data, old_unseen_data):
  target_column = 'OOD'

  if not isinstance(old_unseen_data, pd.DataFrame) or old_unseen_data.empty:
      old_unseen_data = pd.DataFrame(columns=new_valuator_data.columns)
  # Ensure which_data_aquired is correctly sized to match new_valuator_data
  assert len(which_data_aquired) == new_valuator_data.shape[0], "Mismatch in lengths."

  N = sum(which_data_aquired.astype(bool))
  old_training_data_modified = old_training_data.iloc[N:]
  new_train_data = pd.concat([old_training_data_modified, new_valuator_data[which_data_aquired.astype(bool)]])
  new_unseen_data = pd.concat([old_unseen_data, new_valuator_data[~which_data_aquired.astype(bool)]])
  print(f"new_train_data.shape: {new_train_data.shape}, new_unseen_data.shape: {new_unseen_data.shape}")

  print(f"new_train_data.shape: {new_train_data.shape},new_unseen_data.shape: {new_unseen_data.shape}")
  unseen_data_ = new_unseen_data.drop(columns=['install'])
  train_data_ = new_train_data.drop(columns=['install'])

  # Add OOD column
  unseen_data_['OOD'] = 0
  train_data_['OOD'] = 1

  ood_train_data = pd.concat([train_data_, unseen_data_], ignore_index=True)

  preprocessor_OOD = DataPreprocessor(numerical_columns=numerical_columns, target_column=target_column)
  preprocessor_OOD.preprocess_train(ood_train_data)

  # Preprocess your DataFrame
  X = preprocessor_OOD.balanced_df.drop(columns=[target_column])
  y = preprocessor_OOD.balanced_df[target_column]

  # Convert the target to a numeric type, if it's not already
  y = pd.to_numeric(y, errors='coerce')
  input_shape = X.shape[1]

  # Initialize and train the model
  OOD_model = OODModel(input_shape)
  OOD_model.fit(X, y,epochs=20)
  postprocessor_OOD = Postprocessor(OOD_model)

  return Valuator(preprocessor_OOD,OOD_model,postprocessor_OOD), new_train_data, new_unseen_data

Let's add aquired data to TRAIN_DATA,

In [ ]:
valuator_ood, TRAIN_DATA, UNSEEN_DATA = from_valuator_create_ood(aquired_data,TRAFFIC_DATA, TRAIN_DATA, UNSEEN_DATA)

new_train_data.shape: (1571590, 17), new_unseen_data.shape: (442802, 17)
new_train_data.shape: (1571590, 17),new_unseen_data.shape: (442802, 17)
Epoch 1/2
22141/22141 [==============================] - 65s 3ms/step - loss: 0.6629 - accuracy: 0.6307 - val_loss: 0.9654 - val_accuracy: 0.1324
Epoch 2/2
22141/22141 [==============================] - 58s 3ms/step - loss: 0.6471 - accuracy: 0.6378 - val_loss: 0.9607 - val_accuracy: 0.1479


In [ ]:
valuator = given_data_create_valuator(TRAIN_DATA)

Epoch 1/30
911/911 [==============================] - 4s 3ms/step - loss: 0.1677 - binary_crossentropy: 0.1677 - val_loss: 0.0403 - val_binary_crossentropy: 0.0403
Epoch 2/30
911/911 [==============================] - 2s 2ms/step - loss: 0.1175 - binary_crossentropy: 0.1175 - val_loss: 0.0311 - val_binary_crossentropy: 0.0311
Epoch 3/30
911/911 [==============================] - 2s 2ms/step - loss: 0.1052 - binary_crossentropy: 0.1052 - val_loss: 0.0193 - val_binary_crossentropy: 0.0193
Epoch 4/30
911/911 [==============================] - 2s 2ms/step - loss: 0.1029 - binary_crossentropy: 0.1029 - val_loss: 0.0235 - val_binary_crossentropy: 0.0235
Epoch 5/30
911/911 [==============================] - 3s 3ms/step - loss: 0.1007 - binary_crossentropy: 0.1007 - val_loss: 0.0217 - val_binary_crossentropy: 0.0217
Epoch 6/30
911/911 [==============================] - 3s 4ms/step - loss: 0.0995 - binary_crossentropy: 0.0995 - val_loss: 0.0226 - val_binary_crossentropy: 0.0226
Epoch 7/30
911/9

Now we have available OOD and Conversion model to proceed with logic


In [ ]:
TRAFFIC_DATA = daily_dfs[9]

In [ ]:
probs = valuator.predict(TRAFFIC_DATA)[1]

6604/6604 [==============================] - 10s 1ms/step


In [ ]:
probs_ood = valuator_ood.predict(TRAFFIC_DATA)[1]

6604/6604 [==============================] - 10s 1ms/step


dane ze stanow powinny byc odchylone w ocenie valuator_ood. w mc dropout wylosujemy top 100 probek i policzyc wariancje - to jest kryterium. porownanie takiego rankingu vs OOD. Jesli wezmiemy recall@100, to oczekujemy ze ood bedzie mialo wiecej USA

In [ ]:
#With OOD we would select:
Recall = np.hstack([probs,probs_ood,(TRAFFIC_DATA['country']=='US').map(int).values.reshape(-1,1)])
Recall[Recall[:, 1].argsort()][:100][:,2].sum(),sum(Recall[:,2])/len(Recall[:,2])

(24.0, 0.14911487883706295)

In [ ]:
#With MCDropout we'd select:
Recall[Recall[:, 0].argsort()][:100][:,2].sum()

21.0